# Week 3: Exercise Solutions - Social Media & Graph Analytics

**Web and Social Network Analytics**

---

This notebook contains complete solutions for all Week 3 exercises.

## Setup

In [ ]:
# Graph analysis
import networkx as nx
from networkx.algorithms import bipartite
from networkx.algorithms.community import kernighan_lin_bisection

# Visualization
import matplotlib.pyplot as plt

# Data handling
import pandas as pd
import pprint as pp

print('All libraries imported successfully!')

---

## Exercise 1 Solution: Building a Social Network Graph

**Task**: Create a directed graph representing Twitter follows between 6 users.

In [ ]:
# Step 1: Create a directed graph
G = nx.DiGraph()

# Step 2: Add edges representing follows (at least 8 edges)
# alice follows bob, carol, dave
# bob follows alice (mutual), carol
# carol follows dave, eve
# dave follows eve, frank
# eve follows frank
# frank follows alice

G.add_edges_from([
    ('alice', 'bob'),
    ('alice', 'carol'),
    ('alice', 'dave'),
    ('bob', 'alice'),      # Mutual follow with alice
    ('bob', 'carol'),
    ('carol', 'dave'),
    ('carol', 'eve'),
    ('dave', 'eve'),
    ('dave', 'frank'),
    ('eve', 'frank'),
    ('frank', 'alice')     # Creates a cycle back to alice
])

# Step 3: Print basic statistics
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"\nNodes: {list(G.nodes())}")
print(f"\nEdges (who follows whom):")
for u, v in G.edges():
    print(f"  {u} follows {v}")

In [ ]:
# Step 4: Visualize the graph
pos = nx.spring_layout(G, seed=2000)

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightblue',
        font_size=12, font_weight='bold', arrows=True, arrowsize=20,
        edge_color='gray', width=2)
plt.title("Twitter Follows Network", fontsize=14)
plt.show()

In [ ]:
# Additional analysis: In-degree vs Out-degree
print("\nIn-degree (followers) vs Out-degree (following):")
print("-" * 50)
for node in G.nodes():
    print(f"{node:10} | in-degree: {G.in_degree(node):2} | out-degree: {G.out_degree(node):2}")

**Key Points:**
- The graph has 6 nodes and 11 edges
- Alice and Bob have a mutual follow relationship
- There's a cycle: alice -> dave -> frank -> alice
- In directed graphs, in-degree = followers, out-degree = following

---

## Exercise 2 Solution: Calculating Centrality Measures

**Task**: Calculate degree and betweenness centrality.

In [ ]:
# Step 1: Calculate degree centrality
degree = nx.degree_centrality(G)

print("Degree Centrality (normalized by max possible connections):")
print("-" * 50)
for node, value in sorted(degree.items(), key=lambda x: x[1], reverse=True):
    print(f"{node:10}: {value:.3f}")

In [ ]:
# Step 2: Calculate betweenness centrality
betweenness = nx.betweenness_centrality(G)

print("Betweenness Centrality (how often node is on shortest paths):")
print("-" * 50)
for node, value in sorted(betweenness.items(), key=lambda x: x[1], reverse=True):
    print(f"{node:10}: {value:.3f}")

In [ ]:
# Step 3: Find and print the most central nodes
max_degree_node = max(degree, key=degree.get)
max_betweenness_node = max(betweenness, key=betweenness.get)

print(f"Most central by degree: {max_degree_node} (score: {degree[max_degree_node]:.3f})")
print(f"Most central by betweenness: {max_betweenness_node} (score: {betweenness[max_betweenness_node]:.3f})")

In [ ]:
# Step 4: Visualize with node sizes based on degree centrality
sizes = [degree[node] * 3000 for node in G.nodes()]

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_size=sizes, node_color='lightcoral',
        font_size=12, arrows=True, arrowsize=15, width=2)
plt.title("Node Size = Degree Centrality", fontsize=14)
plt.show()

In [ ]:
# Visualize with betweenness centrality
sizes_betweenness = [max(betweenness[node] * 5000, 200) for node in G.nodes()]

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_size=sizes_betweenness, node_color='lightgreen',
        font_size=12, arrows=True, arrowsize=15, width=2)
plt.title("Node Size = Betweenness Centrality", fontsize=14)
plt.show()

**Key Points:**
- Degree centrality measures total connections (in + out in directed graphs)
- Betweenness centrality measures how often a node acts as a bridge
- These measures can give different rankings for "importance"

---

## Exercise 3 Solution: Clustering Coefficient Calculation

**Task**: Calculate clustering coefficients by hand and verify.

In [ ]:
# Step 1: Create the undirected graph
G_cluster = nx.Graph()
G_cluster.add_edges_from([
    ('A', 'B'), ('A', 'C'), ('A', 'D'),
    ('B', 'C'),
    ('C', 'D'),
    ('D', 'E')
])

# Step 2: Visualize
pos_cluster = nx.spring_layout(G_cluster, seed=42)

plt.figure(figsize=(8, 6))
nx.draw(G_cluster, pos_cluster, with_labels=True, node_size=1500, 
        node_color='lightblue', font_size=14, font_weight='bold', width=2)
plt.title("Graph for Clustering Coefficient Calculation")
plt.show()

In [ ]:
# Step 3: Hand calculation for node A
print("=" * 50)
print("HAND CALCULATION FOR NODE A")
print("=" * 50)

neighbors_A = list(G_cluster.neighbors('A'))
print(f"1. Neighbors of A: {neighbors_A}")
print(f"2. Degree of A (k): {len(neighbors_A)}")

# Count edges between neighbors
edges_between_neighbors = 0
neighbor_pairs = []
for i, n1 in enumerate(neighbors_A):
    for n2 in neighbors_A[i+1:]:
        neighbor_pairs.append((n1, n2))
        if G_cluster.has_edge(n1, n2):
            edges_between_neighbors += 1
            print(f"   Edge found: {n1}-{n2}")
        else:
            print(f"   No edge: {n1}-{n2}")

print(f"3. Edges between neighbors: {edges_between_neighbors}")

k = len(neighbors_A)
max_edges = k * (k - 1) // 2
print(f"4. Maximum possible edges: k(k-1)/2 = {k}*{k-1}/2 = {max_edges}")

clustering_A = edges_between_neighbors / max_edges if max_edges > 0 else 0
print(f"5. Clustering coefficient: {edges_between_neighbors}/{max_edges} = {clustering_A:.3f}")

In [ ]:
# Step 4: Hand calculation for node D
print("=" * 50)
print("HAND CALCULATION FOR NODE D")
print("=" * 50)

neighbors_D = list(G_cluster.neighbors('D'))
print(f"1. Neighbors of D: {neighbors_D}")
print(f"2. Degree of D (k): {len(neighbors_D)}")

# Count edges between neighbors
edges_D = 0
for i, n1 in enumerate(neighbors_D):
    for n2 in neighbors_D[i+1:]:
        if G_cluster.has_edge(n1, n2):
            edges_D += 1
            print(f"   Edge found: {n1}-{n2}")
        else:
            print(f"   No edge: {n1}-{n2}")

print(f"3. Edges between neighbors: {edges_D}")

k_D = len(neighbors_D)
max_edges_D = k_D * (k_D - 1) // 2
print(f"4. Maximum possible edges: {k_D}*{k_D-1}/2 = {max_edges_D}")

clustering_D = edges_D / max_edges_D if max_edges_D > 0 else 0
print(f"5. Clustering coefficient: {edges_D}/{max_edges_D} = {clustering_D:.3f}")

In [ ]:
# Step 5: Verify with NetworkX
print("=" * 50)
print("NETWORKX VERIFICATION")
print("=" * 50)

clustering_all = nx.clustering(G_cluster)

for node, coeff in clustering_all.items():
    print(f"Clustering({node}): {coeff:.3f}")

print(f"\nHand calculation for A: {clustering_A:.3f} vs NetworkX: {clustering_all['A']:.3f} - {'MATCH!' if abs(clustering_A - clustering_all['A']) < 0.001 else 'MISMATCH!'}")
print(f"Hand calculation for D: {clustering_D:.3f} vs NetworkX: {clustering_all['D']:.3f} - {'MATCH!' if abs(clustering_D - clustering_all['D']) < 0.001 else 'MISMATCH!'}")

In [ ]:
# Step 6: Calculate average clustering coefficient
avg_clustering = nx.average_clustering(G_cluster)
print(f"\nAverage Clustering Coefficient: {avg_clustering:.3f}")

**Key Points:**
- Node A has clustering 0.667: 2 out of 3 possible neighbor connections exist
- Node D has clustering 0.333: 1 out of 3 possible neighbor connections exist
- Node E has clustering 0.0: only one neighbor, no triangle possible
- Higher clustering = friends know each other (tight-knit group)

---

## Exercise 4 Solution: Community Detection with Kernighan-Lin

**Task**: Apply Kernighan-Lin algorithm to partition a graph.

In [ ]:
# Step 1: Create graph with two communities
G_community = nx.Graph()

# Community 1: A, B, C, D (densely connected - complete subgraph)
G_community.add_edges_from([
    ('A', 'B'), ('A', 'C'), ('A', 'D'),
    ('B', 'C'), ('B', 'D'),
    ('C', 'D')
])

# Community 2: E, F, G, H (densely connected - complete subgraph)
G_community.add_edges_from([
    ('E', 'F'), ('E', 'G'), ('E', 'H'),
    ('F', 'G'), ('F', 'H'),
    ('G', 'H')
])

# Bridge edges between communities (sparse)
G_community.add_edges_from([
    ('D', 'E'),  # One bridge
    ('C', 'F')   # Another bridge
])

print(f"Graph has {G_community.number_of_nodes()} nodes and {G_community.number_of_edges()} edges")

In [ ]:
# Step 2: Visualize the original graph
pos_comm = nx.spring_layout(G_community, seed=42)

# Color by original communities
original_set1 = {'A', 'B', 'C', 'D'}
original_set2 = {'E', 'F', 'G', 'H'}
colors = ['lightblue' if node in original_set1 else 'lightgreen' 
          for node in G_community.nodes()]

plt.figure(figsize=(10, 8))
nx.draw(G_community, pos_comm, with_labels=True, node_size=2000, 
        node_color=colors, font_size=14, font_weight='bold', width=2)
plt.title("Original Communities: Blue (A-D) vs Green (E-H)")
plt.show()

In [ ]:
# Step 3: Calculate original cut size
original_cut = nx.cut_size(G_community, original_set1, original_set2)
print(f"Original partition cut size: {original_cut}")
print(f"  (Number of edges between set1 and set2)")

In [ ]:
# Step 4: Apply Kernighan-Lin bisection
partition = kernighan_lin_bisection(G_community)
partition1, partition2 = partition

print(f"Kernighan-Lin Results:")
print(f"  Partition 1: {partition1}")
print(f"  Partition 2: {partition2}")

In [ ]:
# Step 5: Compare results
kl_cut = nx.cut_size(G_community, partition1, partition2)

print(f"\nComparison:")
print(f"  Original partition cut: {original_cut}")
print(f"  Kernighan-Lin cut: {kl_cut}")

if kl_cut < original_cut:
    print(f"  Kernighan-Lin found a BETTER partition!")
elif kl_cut == original_cut:
    print(f"  Kernighan-Lin found an EQUAL partition.")
else:
    print(f"  Original partition was better.")

In [ ]:
# Step 6: Visualize Kernighan-Lin partition
colors_kl = ['lightblue' if node in partition1 else 'lightgreen' 
             for node in G_community.nodes()]

plt.figure(figsize=(10, 8))
nx.draw(G_community, pos_comm, with_labels=True, node_size=2000, 
        node_color=colors_kl, font_size=14, font_weight='bold', width=2)
plt.title(f"Kernighan-Lin Partition (Cut Size: {kl_cut})")
plt.show()

**Key Points:**
- The original partition (A-D vs E-H) has cut size 2 (edges D-E and C-F)
- Kernighan-Lin should find this same partition since it's optimal
- The algorithm minimizes edges between partitions while balancing partition sizes

---

## Exercise 5 Solution: Analyzing Student Network Data

**Task**: Load and analyze the student familiarity network.

In [ ]:
# Step 1: Load the data
df = pd.read_csv('data/graph_large.csv', index_col=0)

# Get column information
col_names = list(df.columns)
personality_questions = col_names[:10]
people = col_names[-30:]
experiences_questions = col_names[10:-30]

print(f"Dataset shape: {df.shape}")
print(f"Number of personality questions: {len(personality_questions)}")
print(f"Number of experience questions: {len(experiences_questions)}")
print(f"Number of students: {len(people)}")
print(f"\nStudent IDs: {people[:5]}... (showing first 5)")

In [ ]:
# Step 2: Build the directed graph (edges with weight > 3)
DG = nx.DiGraph()

for row, row_values in df.iterrows():
    for column, value in enumerate(row_values):
        col_name = df.columns[column]
        # Only include if it's a person column and value > 3
        if col_name in people and value > 3:
            DG.add_edge(row, col_name, weight=value)

print(f"Graph created:")
print(f"  Nodes: {DG.number_of_nodes()}")
print(f"  Edges: {DG.number_of_edges()}")
print(f"  Density: {nx.density(DG):.3f}")

In [ ]:
# Step 3: Calculate all centrality measures
degree = nx.degree_centrality(DG)
betweenness = nx.betweenness_centrality(DG)
pagerank = nx.pagerank(DG)

In [ ]:
# Step 4: Print top 5 by each measure
def print_top_5(measure_dict, name):
    """Print top 5 nodes by a centrality measure."""
    sorted_items = sorted(measure_dict.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"\n{'='*40}")
    print(f"Top 5 by {name}")
    print(f"{'='*40}")
    for rank, (node, score) in enumerate(sorted_items, 1):
        print(f"  {rank}. {node}: {score:.4f}")

print_top_5(degree, "Degree Centrality")
print_top_5(betweenness, "Betweenness Centrality")
print_top_5(pagerank, "PageRank")
print_top_5(hubs, "Hub Score")
print_top_5(authorities, "Authority Score")

In [ ]:
# Create a comprehensive comparison table
comparison_data = []
for node in DG.nodes():
    comparison_data.append({
        'Student': node,
        'Degree': round(degree[node], 4),
        'Betweenness': round(betweenness[node], 4),
        'PageRank': round(pagerank[node], 4),
        'Hub': round(hubs[node], 4),
        'Authority': round(authorities[node], 4)
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('PageRank', ascending=False)

print("\nTop 10 Students by PageRank (with all measures):")
print(comparison_df.head(10).to_string(index=False))

In [ ]:
# Step 5: Visualize with PageRank-based node sizes
plt.figure(figsize=(14, 12))

pos = nx.spring_layout(DG, seed=42)

# Scale sizes by PageRank
sizes = [pagerank[node] * 10000 for node in DG.nodes()]

nx.draw(DG, pos, with_labels=True, node_size=sizes, node_color='lightcoral',
        font_size=8, arrows=True, arrowsize=10, width=0.5, alpha=0.7)
plt.title("Student Network - Node Size = PageRank", fontsize=14)
plt.show()

**Analysis:**

1. **Which students appear most "important"?**
   - The top students by PageRank tend to be those who are well-known by many others who are themselves well-known
   - Different measures can highlight different students because they measure different aspects of importance

2. **Why might degree and betweenness give different rankings?**
   - **Degree centrality** measures how many connections a student has (popularity)
   - **Betweenness centrality** measures how often a student is a bridge between others
   - A student can be very popular but not be a bridge (everyone in their group knows each other)
   - A student can be a critical bridge with fewer direct connections

3. **Hub vs Authority in this context:**
   - **High hub score**: Student who knows many well-connected students (a social connector)
   - **High authority score**: Student who is known by many social connectors (well-respected)

---

## Bonus Solution: Triadic Closure Analysis

In [ ]:
def find_potential_connections(G, min_common_neighbors=2):
    """
    Find pairs of nodes that might become connected via triadic closure.
    
    Args:
        G: NetworkX graph (undirected)
        min_common_neighbors: Minimum common neighbors to consider
        
    Returns:
        List of tuples: (node1, node2, num_common_neighbors)
    """
    potential = []
    nodes = list(G.nodes())
    
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i+1:]:
            # Skip if already connected
            if G.has_edge(node1, node2):
                continue
            
            # Find common neighbors
            neighbors1 = set(G.neighbors(node1))
            neighbors2 = set(G.neighbors(node2))
            common = neighbors1 & neighbors2
            
            if len(common) >= min_common_neighbors:
                potential.append((node1, node2, len(common)))
    
    # Sort by number of common neighbors (descending)
    potential.sort(key=lambda x: x[2], reverse=True)
    
    return potential

In [ ]:
# Test with the student network (convert to undirected)
G_undirected = DG.to_undirected()

potential_connections = find_potential_connections(G_undirected, min_common_neighbors=2)

print(f"Found {len(potential_connections)} potential connections via triadic closure")
print(f"\nTop 10 potential connections:")
print("-" * 50)
for node1, node2, common in potential_connections[:10]:
    print(f"{node1} - {node2}: {common} common neighbors")

**Key Point**: Triadic closure predicts that students with many mutual friends are likely to become connected. This is the basis for "People You May Know" features in social networks.